# Measles Data Cleaning

### Importar librerías y cargar datos

Importamos pandas y cargamos los archivos `cases_month.csv` y `cases_year.csv` en DataFrames.

In [ ]:
!pip install pandas numpy


El sistema no puede encontrar la ruta especificada.


In [77]:
import pandas as pd
import numpy as np

cases_month = pd.read_csv('cases_month.csv')
cases_year = pd.read_csv('cases_year.csv')

### Visualizar primeras filas de los DataFrames

Mostramos las primeras filas de `cases_month` y `cases_year` para explorar la estructura inicial de los datos.

In [78]:
print("cases_month:")
display(cases_month.head())

print("\ncases_year:")
display(cases_year.head())

cases_month:


,region,country,iso3,year,month,measles_suspect,measles_clinical,measles_epi_linked,measles_lab_confirmed,measles_total,rubella_clinical,rubella_epi_linked,rubella_lab_confirmed,rubella_total,discarded
0,AFR,Algeria,DZA,2012,1,8.0,6.0,0.0,2.0,8.0,NaN,NaN,NaN,NaN,0.0
1,AFR,Algeria,DZA,2012,2,10.0,10.0,0.0,0.0,10.0,NaN,NaN,NaN,NaN,0.0
2,AFR,Algeria,DZA,2012,3,17.0,17.0,0.0,0.0,17.0,NaN,NaN,NaN,NaN,0.0
3,AFR,Algeria,DZA,2012,4,7.0,5.0,0.0,0.0,5.0,0.0,0.0,1.0,1.0,2.0
4,AFR,Algeria,DZA,2012,5,14.0,11.0,0.0,0.0,11.0,0.0,0.0,3.0,3.0,3.0



cases_year:


,region,country,iso3,year,total_population,annualized_population_most_recent_year_only,total_suspected_measles_rubella_cases,measles_total,measles_lab_confirmed,measles_epi_linked,measles_clinical,measles_incidence_rate_per_1000000_total_population,rubella_total,rubella_lab_confirmed,rubella_epi_linked,rubella_clinical,rubella_incidence_rate_per_1000000_total_population,discarded_cases,discarded_non_measles_rubella_cases_per_100000_total_population
0,AFRO,Algeria,DZA,2012,37646166,37646166,76.0,55,2,0,53,1.46,13,13,0,0,0.35,8.0,0.02
1,AFRO,Algeria,DZA,2013,38414172,38414172,85.0,0,0,0,0,0.00,29,29,0,0,0.75,56.0,0.15
2,AFRO,Algeria,DZA,2014,39205031,39205031,49.0,0,0,0,0,0.00,3,3,0,0,0.08,46.0,0.12
3,AFRO,Algeria,DZA,2015,40019529,40019529,109.0,62,2,60,0,1.55,2,2,0,0,0.05,45.0,0.11
4,AFRO,Algeria,DZA,2016,40850721,40850721,93.0,49,21,27,1,1.20,11,11,0,0,0.27,33.0,0.08


### Inspección inicial

In [79]:
cases_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22780 entries, 0 to 22779
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   region                 22780 non-null  object 
 1   country                22780 non-null  object 
 2   iso3                   22780 non-null  object 
 3   year                   22780 non-null  int64  
 4   month                  22780 non-null  int64  
 5   measles_suspect        22632 non-null  float64
 6   measles_clinical       22632 non-null  float64
 7   measles_epi_linked     22632 non-null  float64
 8   measles_lab_confirmed  22632 non-null  float64
 9   measles_total          22632 non-null  float64
 10  rubella_clinical       7811 non-null   float64
 11  rubella_epi_linked     7811 non-null   float64
 12  rubella_lab_confirmed  7811 non-null   float64
 13  rubella_total          7811 non-null   float64
 14  discarded              22632 non-null  float64
dtypes:

In [80]:
cases_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2382 entries, 0 to 2381
Data columns (total 19 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   region                                                           2382 non-null   object 
 1   country                                                          2380 non-null   object 
 2   iso3                                                             2382 non-null   object 
 3   year                                                             2382 non-null   int64  
 4   total_population                                                 2382 non-null   int64  
 5   annualized_population_most_recent_year_only                      2382 non-null   int64  
 6   total_suspected_measles_rubella_cases                            2295 non-null   float64
 7   measles_total                             

In [81]:
# Agrupar por country y year en cases_month y sumar measles_total
measles_monthly_sum = cases_month.groupby(['country', 'year'])['measles_total'].sum().reset_index(name='measles_total_month_sum')

# Extraer measles_total por country y year de cases_year
measles_yearly = cases_year[['country', 'year', 'measles_total']]

# Unir ambos resultados para comparar
comparison = pd.merge(
    measles_monthly_sum,
    measles_yearly,
    on=['country', 'year'],
    how='inner',
    suffixes=('_month', '_year')
)

display(comparison)

,country,year,measles_total_month_sum,measles_total
0,Afghanistan,2012,2791.0,2791
1,Afghanistan,2013,430.0,430
2,Afghanistan,2014,552.0,552
3,Afghanistan,2015,1191.0,1191
4,Afghanistan,2016,641.0,641
...,...,...,...,...
2375,Zimbabwe,2021,0.0,0
2376,Zimbabwe,2022,5152.0,5152
2377,Zimbabwe,2023,17.0,17
2378,Zimbabwe,2024,36.0,36


In [82]:
print("Valores únicos en 'country':", cases_month['country'].nunique())
print("Valores únicos en 'year':", cases_month['year'].nunique())
print("Valores únicos en 'month':", cases_month['month'].nunique())

Valores únicos en 'country': 193
Valores únicos en 'year': 14
Valores únicos en 'month': 12


In [83]:
cases_month['month'].unique()

array([ 1,  2,  3,  4,  5,  6,  7, 10, 11, 12,  8,  9], dtype=int64)

### Conversión de tipos para columnas numéricas

In [84]:
for col in [
    "total_population",
    "annualized_population_most_recent_year_only",
    "total_suspected_measles_rubella_cases",
    "measles_total",
    "measles_lab_confirmed",
    "measles_epi_linked",
    "measles_clinical",
    "measles_incidence_rate_per_1000000_total_population",
    "rubella_total",
    "rubella_lab_confirmed",
    "rubella_epi_linked",
    "rubella_clinical",
    "rubella_incidence_rate_per_1000000_total_population",
    "discarded_cases",
    "discarded_non_measles_rubella_cases_per_100000_total_population"
]:
    cases_year[col] = pd.to_numeric(cases_year[col], errors="coerce")

In [85]:
for col in [
    "year", "month", "measles_suspect", "measles_clinical",
    "measles_epi_linked", "measles_lab_confirmed", "measles_total",
    "rubella_clinical", "rubella_epi_linked", "rubella_lab_confirmed",
    "rubella_total", "discarded"
]:
    cases_month[col] = pd.to_numeric(cases_month[col], errors="coerce")

## Inspección de negativos

In [86]:
# Solo seleccionamos columnas numéricas
num_cols = cases_month.select_dtypes(include=["number"]).columns

# Verificación de valores negativos por columna
negativos = (cases_month[num_cols] < 0).sum()
negativos = negativos[negativos > 0]

print("▶️ Columnas con valores negativos:")
display(negativos)

# Mostrar algunas filas donde al menos una columna numérica tenga valor negativo
print("\n▶️ Ejemplos de registros con valores negativos:")
filas_con_negativos = cases_month[(cases_month[num_cols] < 0).any(axis=1)]
display(filas_con_negativos.head())

▶️ Columnas con valores negativos:


measles_clinical    14
measles_total        6
dtype: int64


▶️ Ejemplos de registros con valores negativos:


,region,country,iso3,year,month,measles_suspect,measles_clinical,measles_epi_linked,measles_lab_confirmed,measles_total,rubella_clinical,rubella_epi_linked,rubella_lab_confirmed,rubella_total,discarded
18928,SEAR,Bhutan,BTN,2014,8,14.0,-14.0,0.0,0.0,-14.0,0.0,0.0,0.0,0.0,28.0
19364,SEAR,Indonesia,IDN,2016,7,294.0,-15.0,0.0,57.0,42.0,0.0,0.0,4.0,4.0,252.0
19365,SEAR,Indonesia,IDN,2016,8,922.0,-131.0,0.0,126.0,-5.0,0.0,0.0,4.0,4.0,927.0
19366,SEAR,Indonesia,IDN,2016,9,685.0,-68.0,0.0,37.0,-31.0,0.0,0.0,4.0,4.0,716.0
19367,SEAR,Indonesia,IDN,2016,10,296.0,-204.0,0.0,9.0,-195.0,0.0,0.0,0.0,0.0,491.0


## Correción de negativos

In [87]:
def limpiar_negativos(df):
    num_cols = df.select_dtypes(include=["number"]).columns
    df[num_cols] = df[num_cols].applymap(lambda x: x if x >= 0 else np.nan)
    return df

cases_month = limpiar_negativos(cases_month)

# Verificación posterior a la limpieza
print("\n✅ Verificación tras la limpieza:")
display((cases_month[num_cols] < 0).sum()[(cases_month[num_cols] < 0).sum() > 0])


✅ Verificación tras la limpieza:


Series([], dtype: int64)

### Validación de valores nulos y duplicados

In [88]:
print("Valores nulos cases_month:")
print(cases_month.isna().sum())
print("Valores nulos cases_year:")
print(cases_year.isna().sum())

print("Duplicados cases_month:", cases_month.duplicated().sum())
print("Duplicados cases_year:", cases_year.duplicated().sum())

Valores nulos cases_month:
region                       0
country                      0
iso3                         0
year                         0
month                        0
measles_suspect            148
measles_clinical           162
measles_epi_linked         148
measles_lab_confirmed      148
measles_total              154
rubella_clinical         14969
rubella_epi_linked       14969
rubella_lab_confirmed    14969
rubella_total            14969
discarded                  148
dtype: int64
Valores nulos cases_year:
region                                                               0
country                                                              2
iso3                                                                 0
year                                                                 0
total_population                                                     0
annualized_population_most_recent_year_only                          0
total_suspected_measles_rubella_cases      

### Resumen preliminar

#### Sobre cases_month:

22780 registros

~148 valores nulos en measles (sospechosos, clínicos, etc.)

~14.969 valores nulos en rubella → esto es normal, muchos países no reportan rubella mensual

Sin duplicados

En principio los valores nulos en rubella no son graves: simplemente hay países que no reportan casos cada mes → imputación no necesaria, se pueden mantener como NaN para indicar ausencia de dato.

#### Sobre cases_year:

2382 registros

2 países sin nombre (country con null)

87 valores nulos en total_suspected_measles_rubella_cases y en discarded_cases

408 valores nulos en discarded_non_measles_rubella_cases_per_100000_total_population

Sin duplicados

Estos campos nulos también reflejan datos que no fueron reportados por el país → podemos mantenerlos como nulos inicialmente para preservar la integridad del reporte.

Podemos decidir si en visualización tratamos esos valores (por ejemplo colorear “sin dato”), pero no imputarlos artificialmente.

### Eliminar los 2 registros con country nulo en cases_year

In [89]:
cases_year[cases_year['country'].isna()]

,region,country,iso3,year,total_population,annualized_population_most_recent_year_only,total_suspected_measles_rubella_cases,measles_total,measles_lab_confirmed,measles_epi_linked,measles_clinical,measles_incidence_rate_per_1000000_total_population,rubella_total,rubella_lab_confirmed,rubella_epi_linked,rubella_clinical,rubella_incidence_rate_per_1000000_total_population,discarded_cases,discarded_non_measles_rubella_cases_per_100000_total_population
1212,EMRO,NaN,SSD,2012,10756240,10756240,2050.0,1969,140,20,1809,183.06,4,4,0,0,0.37,76.0,0.71
1213,EMRO,NaN,SSD,2013,11166276,11166276,620.0,520,157,13,350,46.57,25,25,0,0,2.24,5.0,0.04


In [90]:
cases_year[cases_year['iso3'] == 'SSD']

,region,country,iso3,year,total_population,annualized_population_most_recent_year_only,total_suspected_measles_rubella_cases,measles_total,measles_lab_confirmed,measles_epi_linked,measles_clinical,measles_incidence_rate_per_1000000_total_population,rubella_total,rubella_lab_confirmed,rubella_epi_linked,rubella_clinical,rubella_incidence_rate_per_1000000_total_population,discarded_cases,discarded_non_measles_rubella_cases_per_100000_total_population
519,AFRO,South Sudan,SSD,2014,11235073,11235073,467.0,466,0,0,466,41.48,1,1,0,0,0.09,0.0,0.00
520,AFRO,South Sudan,SSD,2015,11107562,11107562,536.0,366,56,79,231,32.95,47,47,0,0,4.23,123.0,1.11
521,AFRO,South Sudan,SSD,2016,10830102,10830102,1120.0,902,138,20,744,83.29,31,31,0,0,2.86,187.0,1.73
522,AFRO,South Sudan,SSD,2017,10259154,10259154,692.0,487,87,163,237,47.47,71,71,0,0,6.92,134.0,1.31
523,AFRO,South Sudan,SSD,2018,10122977,10122977,267.0,127,25,8,94,12.55,44,44,0,0,4.35,96.0,0.95
524,AFRO,South Sudan,SSD,2019,10423384,10423384,1893.0,1472,144,1173,155,141.22,29,29,0,0,2.78,392.0,3.76
525,AFRO,South Sudan,SSD,2020,10698467,10698467,1326.0,1204,80,1103,21,112.54,45,45,0,0,4.21,77.0,0.72
526,AFRO,South Sudan,SSD,2021,10865780,10865780,252.0,130,7,4,119,11.96,20,20,0,0,1.84,102.0,0.94
527,AFRO,South Sudan,SSD,2022,11021178,11021178,2301.0,1808,291,1443,74,164.05,159,159,0,0,14.43,317.0,2.88
528,AFRO,South Sudan,SSD,2023,11483374,11483374,863.0,547,516,0,31,47.63,42,42,0,0,3.66,274.0,2.39


In [91]:
# Agrupar mensual
measles_monthly_sum = (
    cases_month.groupby(['iso3', 'year'])['measles_total']
    .sum()
    .reset_index(name='measles_total_month_sum')
)

# Extraer anual
measles_yearly = cases_year[['iso3', 'year', 'measles_total']]

# Merge por iso3
comparison_iso3 = pd.merge(
    measles_monthly_sum,
    measles_yearly,
    on=['iso3', 'year'],
    how='outer',  # para detectar faltantes
    suffixes=('_month', '_year')
)

# Ver South Sudan
comparison_iso3[comparison_iso3['iso3'] == 'SSD']


,iso3,year,measles_total_month_sum,measles_total
1990,SSD,2012,1969.0,1969
1991,SSD,2013,520.0,520
1992,SSD,2014,466.0,466
1993,SSD,2015,366.0,366
1994,SSD,2016,902.0,902
1995,SSD,2017,487.0,487
1996,SSD,2018,127.0,127
1997,SSD,2019,1472.0,1472
1998,SSD,2020,1204.0,1204
1999,SSD,2021,130.0,130


In [92]:
# Completar country
cases_year.loc[
    (cases_year['iso3'] == 'SSD') & (cases_year['country'].isna()),
    'country'
] = 'South Sudan'

# Corregir región
cases_year.loc[
    (cases_year['iso3'] == 'SSD') & (cases_year['region'] == 'EMRO'),
    'region'
] = 'AFRO'

In [93]:
cases_year[cases_year['iso3'] == 'SSD']

,region,country,iso3,year,total_population,annualized_population_most_recent_year_only,total_suspected_measles_rubella_cases,measles_total,measles_lab_confirmed,measles_epi_linked,measles_clinical,measles_incidence_rate_per_1000000_total_population,rubella_total,rubella_lab_confirmed,rubella_epi_linked,rubella_clinical,rubella_incidence_rate_per_1000000_total_population,discarded_cases,discarded_non_measles_rubella_cases_per_100000_total_population
519,AFRO,South Sudan,SSD,2014,11235073,11235073,467.0,466,0,0,466,41.48,1,1,0,0,0.09,0.0,0.00
520,AFRO,South Sudan,SSD,2015,11107562,11107562,536.0,366,56,79,231,32.95,47,47,0,0,4.23,123.0,1.11
521,AFRO,South Sudan,SSD,2016,10830102,10830102,1120.0,902,138,20,744,83.29,31,31,0,0,2.86,187.0,1.73
522,AFRO,South Sudan,SSD,2017,10259154,10259154,692.0,487,87,163,237,47.47,71,71,0,0,6.92,134.0,1.31
523,AFRO,South Sudan,SSD,2018,10122977,10122977,267.0,127,25,8,94,12.55,44,44,0,0,4.35,96.0,0.95
524,AFRO,South Sudan,SSD,2019,10423384,10423384,1893.0,1472,144,1173,155,141.22,29,29,0,0,2.78,392.0,3.76
525,AFRO,South Sudan,SSD,2020,10698467,10698467,1326.0,1204,80,1103,21,112.54,45,45,0,0,4.21,77.0,0.72
526,AFRO,South Sudan,SSD,2021,10865780,10865780,252.0,130,7,4,119,11.96,20,20,0,0,1.84,102.0,0.94
527,AFRO,South Sudan,SSD,2022,11021178,11021178,2301.0,1808,291,1443,74,164.05,159,159,0,0,14.43,317.0,2.88
528,AFRO,South Sudan,SSD,2023,11483374,11483374,863.0,547,516,0,31,47.63,42,42,0,0,3.66,274.0,2.39


### Revisar columnas categóricas (por ejemplo region, iso3) para ver si hay inconsistencias de codificación:

In [94]:
print(cases_month['region'].unique())
print(cases_year['region'].unique())

['AFR' 'AMR' 'EMR' 'EUR' 'SEAR' 'WPR']
['AFRO' 'AMRO' 'EMRO' 'EURO' 'SEARO' 'WPRO']


En cases_month las regiones usan códigos de 3 letras tipo:

AFR, AMR, EMR, EUR, SEAR, WPR

En cases_year en cambio usan:

AFRO, AMRO, EMRO, EURO, SEARO, WPRO

Esto significa que para hacer cualquier unión, agregación por continente o visualización conjunta, es clave estandarizar la codificación de región.



In [95]:
# Verificar consistencia interna entre columnas
measles_components = (
    cases_month["measles_clinical"].fillna(0) +
    cases_month["measles_epi_linked"].fillna(0) +
    cases_month["measles_lab_confirmed"].fillna(0)
)

inconsistencias = cases_month[measles_components != cases_month["measles_total"].fillna(0)]
print(f"Número de filas con inconsistencia aritmética en sarampión: {len(inconsistencias)}")
display(inconsistencias.head())



Número de filas con inconsistencia aritmética en sarampión: 11


,region,country,iso3,year,month,measles_suspect,measles_clinical,measles_epi_linked,measles_lab_confirmed,measles_total,rubella_clinical,rubella_epi_linked,rubella_lab_confirmed,rubella_total,discarded
19364,SEAR,Indonesia,IDN,2016,7,294.0,NaN,0.0,57.0,42.0,0.0,0.0,4.0,4.0,252.0
19365,SEAR,Indonesia,IDN,2016,8,922.0,NaN,0.0,126.0,NaN,0.0,0.0,4.0,4.0,927.0
19366,SEAR,Indonesia,IDN,2016,9,685.0,NaN,0.0,37.0,NaN,0.0,0.0,4.0,4.0,716.0
19367,SEAR,Indonesia,IDN,2016,10,296.0,NaN,0.0,9.0,NaN,0.0,0.0,0.0,0.0,491.0
19918,SEAR,Sri Lanka,LKA,2013,1,22.0,NaN,0.0,17.0,4.0,0.0,0.0,2.0,2.0,18.0


In [96]:
error_absoluto = (measles_components - cases_month["measles_total"].fillna(0)).abs()
print(f"Error absoluto medio: {error_absoluto.mean():.2f}")


Error absoluto medio: 0.02


In [97]:
cases_year_clean = cases_year

# Definir correspondencia
region_map = {
    "AFRO": "AFR",
    "AMRO": "AMR",
    "EMRO": "EMR",
    "EURO": "EUR",
    "SEARO": "SEAR",
    "WPRO": "WPR"
}

# Aplicar a cases_year
cases_year_clean['region'] = cases_year_clean['region'].replace(region_map)

In [98]:
print(cases_year_clean['region'].unique())

['AFR' 'AMR' 'EMR' 'EUR' 'SEAR' 'WPR']


In [99]:
print("Valores nulos cases_year_clean:")
print(cases_year_clean.isna().sum())
print("Duplicados cases_year_clean:", cases_year_clean.duplicated().sum())

Valores nulos cases_year_clean:
region                                                               0
country                                                              0
iso3                                                                 0
year                                                                 0
total_population                                                     0
annualized_population_most_recent_year_only                          0
total_suspected_measles_rubella_cases                               87
measles_total                                                        0
measles_lab_confirmed                                                0
measles_epi_linked                                                   0
measles_clinical                                                     0
measles_incidence_rate_per_1000000_total_population                  0
rubella_total                                                        0
rubella_lab_confirmed                        

In [100]:
cases_month_clean = cases_month

In [101]:
cases_month.to_csv("cases_month_clean.csv", index=False)
cases_year.to_csv("cases_year_clean.csv", index=False)